In [15]:
import numpy as np
from paretoset import paretoset
import pandas as pd
import glob
import os
import re


path = "/Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/"
all_files = glob.glob(os.path.join(path, "*.csv"))

quality_metric = 'xpsnr'
# Colormap
cmap = plt.get_cmap('viridis')  # Choose a colormap, e.g., 'viridis'
strings_to_exclude = {path + "test0124.csv", path + "test0137.csv", path + "test0150.csv", path + "test0319.csv", path + "test0370.csv", path + "test0531.csv", path + "test0539.csv", path + "test0797.csv", path + "test0941.csv"}


# Function to extract the desired part from a file path within the range test0001 to test0100
def extract_part(file_path):
    filename = os.path.basename(file_path)
    match = re.search(r'test(0[0-9]{3}|00[1-9][0-9]{1}|0[1-9][0-9]{2}|1000)\.csv$', filename)
    if match:
        return match.group(0).replace('.csv', '')
    else:
        return None

for unique_video_name in all_files:
    if unique_video_name not in strings_to_exclude:
        print("Processing Video: ", unique_video_name)
       
        # The columns of the csv file with the TRQE measurements are:
        # test_cfg,video_name,resolution,qp,bitrate,enc_time,dec_time,PSNRY,PSNRU,PSNRV,VMAF,XPSNRY,XPSNRU,XPSNRV,MSSIMY,MSSIMU,MSSIMV,xpsnr
        sel_data = pd.read_csv(unique_video_name) 
        sel_data['resolution'] = sel_data['resolution'].str.replace('p', '').astype(int)

        arrays = sel_data.to_numpy()

        # # # # # # # # # # # # # # # # # #
        Extract the QT Pareto Front - xpsnr is column 17 and dec_time is 6
        # # # # # # # # # # # # # # # # # #
        QTobj = sel_data.iloc[:, [6, 17]].to_numpy().astype(float)

        mask_qt = paretoset(QTobj, sense=["min", "max"])
        paretoset_QT = QTobj[mask_qt]
        paretoF_QT = arrays[mask_qt]

        df_QT = pd.DataFrame(paretoF_QT,columns=sel_data.columns)
        # Save the DataFrame to a CSV file   
        csv_file_path = path + "CH_QT/chData/" + extract_part(unique_video_name)  + ".csv"
        df_QT.to_csv(csv_file_path, index=False)
      

        # # # # # # # # # # # # # # # # # # #
        # Extract the RQ Pareto Front - ground truth - xpsnr is column 17 and bitrate is 4
        # # # # # # # # # # # # # # # # # # #
        RQobj =  sel_data.iloc[:, [4, 17]].to_numpy().astype(float)

        mask_rq = paretoset(RQobj, sense=["min", "max"])
        paretoset_RQ = RQobj[mask_rq]
        paretoF_RQ = arrays[mask_rq]

        df_RQ = pd.DataFrame(paretoF_RQ,columns=sel_data.columns)
        # Save the DataFrame to a CSV file   
        csv_file_path = path + "CH_RQ/chData/" + extract_part(unique_video_name)  + ".csv"
        df_RQ.to_csv(csv_file_path, index=False)


        # # # # # # # # # # # # # # # # # # #
        # Extract the TRQ Pareto Front
        # wTR = alpha*T + beta*lambda
        # # # # # # # # # # # # # # # # # # #
        TRQobj_ = np.zeros((len(arrays), 2))
        wTR = 0.25*np.log10(sel_data.iloc[:, 6].to_numpy().astype(float)) + 0.75*np.log10(sel_data.iloc[:,4].to_numpy().astype(float))
        TRQobj_[:,0] = wTR 
        TRQobj_[:,1] = arrays[:,17]

        mask_trq = paretoset(TRQobj_, sense=["min", "max"])
        paretoset_TRQ = TRQobj_[mask_trq]
        paretoF = arrays[mask_trq]
        # print(paretoF)

        df_TRQ = pd.DataFrame(paretoF, columns=sel_data.columns)

        # Save the DataFrame to a CSV file   
        csv_file_path = path + "CH_TRQ/chData_025T075R/" + extract_part(unique_video_name)  + ".csv"
        df_TRQ.to_csv(csv_file_path, index=False)

       

Processing Video:  /Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/test0448.csv
Processing Video:  /Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/test0460.csv
Processing Video:  /Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/test0306.csv
Processing Video:  /Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/test0312.csv
Processing Video:  /Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/test0474.csv
Processing Video:  /Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/test0138.csv
Processing Video:  /Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/test0886.csv
Processing Video:  /Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/test0892.csv
Processing Video:  /Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/test0104.csv
Processing Video:  /Users/ak15731/Dow

In [4]:
import pandas as pd
import numpy as np
import os


# Build the Quality-Decoding Time bitrate ladders
# Fixed Static ladder (resolution, kbps): [(360, 145),  (360, 300), (540, 600), (540, 900), (540, 1600), (720, 2400), (720, 3400), (1080, 4500), (1080, 5800), (1440, 8100), (2160, 11600), (2160, 16800)]
rung_rates = [145, 300, 600, 900, 1600, 2400, 3400, 4500, 5800, 8100, 11600, 16800]
delta_rate = 0.25

path = "/Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/"
# Step 1: Define the path to your CSV files
file_path = path + "CH_QT/chData/"  # Adjust the path as needed


# Create a bitrate ladder based on 
# List all filenames in the directory of CH_RQ
# filenames = os.listdir(directory_path)
filenames = [f for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f)) and f.endswith('.csv')]
sorted_filenames = sorted(filenames)

all_ladders = pd.DataFrame()


def ladder_construction(video_name, rung_rates, delta_rate, cur_PF_res, ladder_data):

    for rung in rung_rates:
            # Define the condition
            condition = (cur_PF_res['bitrate'] >= rung) & (cur_PF_res['bitrate'] < rung * (1+delta_rate))

            # Use np.argwhere to find the indices where the condition is True
            pf_rung = np.argwhere(condition.values).flatten()

            if len(pf_rung)>1:
                pf_rung_final = [np.min(pf_rung)]
            else:
                pf_rung_final = pf_rung

            cur_ladder = pd.DataFrame({ "video_name": video_name,
                                        "target_bitrate": rung,
                                        "resolution": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('resolution')],
                                        "selected_bitrate": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('bitrate')],
                                        "PSNRY": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRY')],
                                        "PSNRU": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRU')],
                                        "PSNRV": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRV')],
                                        "PSNR" : (6 * cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRY')] + cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRU')] + cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRV')]) / 8,
                                        "XPSNRY": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('XPSNRY')],
                                        "XPSNRU": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('XPSNRU')],
                                        "XPSNRV": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('XPSNRV')],
                                        "XPSNR" : cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('xpsnr')],
                                        "VMAF": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('VMAF')],
                                        "enc_time": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('enc_time')],
                                        "dec_time": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('dec_time')]
            })
            
            # Concatenate all DataFrames into a single DataFrame
            ladder_data = pd.concat([ladder_data,cur_ladder], ignore_index=True)
    return ladder_data


# Step 2: Read all CSV PF files and sample based on the rate
for file in sorted_filenames:
    ladder_data = pd.DataFrame()
    # print(file)
    cur_PF_res = pd.DataFrame(pd.read_csv(file_path + file))
    
    ladder_data = ladder_construction(file,rung_rates, delta_rate, cur_PF_res, ladder_data)
 
    all_ladders = pd.concat([all_ladders,ladder_data], ignore_index=True)

csv_filename = path + "All_QT_Ladder.csv"
all_ladders.to_csv(csv_filename, index=False)



In [3]:
import pandas as pd
import numpy as np
import os


# Build the Rate-Quality bitrate ladders following the Netflix Dynamic Optimiser Method but based on XPSNR-Rate Curves
# Fixed Static ladder (resolution, kbps): [(360, 145),  (360, 300), (540, 600), (540, 900), (540, 1600), (720, 2400), (720, 3400), (1080, 4500), (1080, 5800), (1440, 8100), (2160, 11600), (2160, 16800)]
rung_rates = [145, 300, 600, 900, 1600, 2400, 3400, 4500, 5800, 8100, 11600, 16800]
delta_rate = 0.25

path = "/Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/"
# Step 1: Define the path to your CSV files
file_path = path + "CH_RQ/chData/"  # Adjust the path as needed


# Create a bitrate ladder based on 
# List all filenames in the directory of CH_RQ
# filenames = os.listdir(directory_path)
filenames = [f for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f)) and f.endswith('.csv')]
sorted_filenames = sorted(filenames)

all_ladders = pd.DataFrame()


def ladder_construction(video_name, rung_rates, delta_rate, cur_PF_res, ladder_data):

    for rung in rung_rates:
            # Define the condition
            condition = (cur_PF_res['bitrate'] >= rung) & (cur_PF_res['bitrate'] < rung * (1+delta_rate))

            # Use np.argwhere to find the indices where the condition is True
            pf_rung = np.argwhere(condition.values).flatten()

            if len(pf_rung)>1:
                pf_rung_final = [np.min(pf_rung)]
            else:
                pf_rung_final = pf_rung

            cur_ladder = pd.DataFrame({ "video_name": video_name,
                                        "target_bitrate": rung,
                                        "resolution": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('resolution')],
                                        "selected_bitrate": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('bitrate')],
                                        "PSNRY": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRY')],
                                        "PSNRU": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRU')],
                                        "PSNRV": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRV')],
                                        "PSNR" : (6 * cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRY')] + cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRU')] + cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRV')]) / 8,
                                        "XPSNRY": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('XPSNRY')],
                                        "XPSNRU": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('XPSNRU')],
                                        "XPSNRV": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('XPSNRV')],
                                        "XPSNR" : cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('xpsnr')],
                                        "VMAF": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('VMAF')],
                                        "enc_time": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('enc_time')],
                                        "dec_time": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('dec_time')]
            })
            
            # Concatenate all DataFrames into a single DataFrame
            ladder_data = pd.concat([ladder_data,cur_ladder], ignore_index=True)
    return ladder_data


# Step 2: Read all CSV PF files and sample based on the rate
for file in sorted_filenames:
    ladder_data = pd.DataFrame()
    # print(file)
    cur_PF_res = pd.DataFrame(pd.read_csv(file_path + file))
    
    ladder_data = ladder_construction(file,rung_rates, delta_rate, cur_PF_res, ladder_data)
 
    all_ladders = pd.concat([all_ladders,ladder_data], ignore_index=True)

csv_filename = path + "All_RQ_Ladder.csv"
all_ladders.to_csv(csv_filename, index=False)



test0001.csv
test0002.csv
test0003.csv
test0004.csv
test0005.csv
test0006.csv
test0007.csv
test0008.csv
test0009.csv
test0010.csv
test0011.csv
test0012.csv
test0013.csv
test0014.csv
test0015.csv
test0016.csv
test0017.csv
test0018.csv
test0019.csv
test0020.csv
test0021.csv
test0022.csv
test0023.csv
test0024.csv
test0025.csv
test0026.csv
test0027.csv
test0028.csv
test0029.csv
test0030.csv
test0031.csv
test0032.csv
test0033.csv
test0034.csv
test0035.csv
test0036.csv
test0037.csv
test0038.csv
test0039.csv
test0040.csv
test0041.csv
test0042.csv
test0043.csv
test0044.csv
test0045.csv
test0046.csv
test0047.csv
test0048.csv
test0049.csv
test0050.csv
test0051.csv
test0052.csv
test0053.csv
test0054.csv
test0055.csv
test0056.csv
test0057.csv
test0058.csv
test0059.csv
test0060.csv
test0061.csv
test0062.csv
test0063.csv
test0064.csv
test0065.csv
test0066.csv
test0067.csv
test0068.csv
test0069.csv
test0070.csv
test0071.csv
test0072.csv
test0073.csv
test0074.csv
test0075.csv
test0076.csv
test0077.csv

In [14]:
import pandas as pd
import numpy as np
import os


# Build the Rate-Quality bitrate ladders following the Netflix Dynamic Optimiser Method but based on XPSNR-Rate Curves
# Fixed Static ladder (resolution, kbps): [(360, 145),  (360, 300), (540, 600), (540, 900), (540, 1600), (720, 2400), (720, 3400), (1080, 4500), (1080, 5800), (1440, 8100), (2160, 11600), (2160, 16800)]
rung_rates = [145, 300, 600, 900, 1600, 2400, 3400, 4500, 5800, 8100, 11600, 16800]
delta_rate = 0.25

path = "/Users/ak15731/Downloads/fastest_energy_measurements/vvenc_slower/noInterp/"
# Step 1: Define the path to your CSV files
file_path = path + "CH_TRQ/chData_075T025R/"  # Adjust the path as needed


# Create a bitrate ladder based on 
# List all filenames in the directory of CH_RQ
# filenames = os.listdir(directory_path)
filenames = [f for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f)) and f.endswith('.csv')]
sorted_filenames = sorted(filenames)

all_ladders = pd.DataFrame()


def ladder_construction(video_name, rung_rates, delta_rate, cur_PF_res, ladder_data):

    for rung in rung_rates:
            # Define the condition
            condition = (cur_PF_res['bitrate'] >= rung) & (cur_PF_res['bitrate'] < rung * (1+delta_rate))

            # Use np.argwhere to find the indices where the condition is True
            pf_rung = np.argwhere(condition.values).flatten()

            if len(pf_rung)>1:
                pf_rung_final = [np.min(pf_rung)]
            else:
                pf_rung_final = pf_rung

            cur_ladder = pd.DataFrame({ "video_name": video_name,
                                        "target_bitrate": rung,
                                        "resolution": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('resolution')],
                                        "selected_bitrate": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('bitrate')],
                                        "PSNRY": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRY')],
                                        "PSNRU": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRU')],
                                        "PSNRV": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRV')],
                                        "PSNR" : (6 * cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRY')] + cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRU')] + cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('PSNRV')]) / 8,
                                        "XPSNRY": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('XPSNRY')],
                                        "XPSNRU": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('XPSNRU')],
                                        "XPSNRV": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('XPSNRV')],
                                        "XPSNR" : cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('xpsnr')],
                                        "VMAF": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('VMAF')],
                                        "enc_time": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('enc_time')],
                                        "dec_time": cur_PF_res.iloc[pf_rung_final, cur_PF_res.columns.get_loc('dec_time')]
            })
            
            # Concatenate all DataFrames into a single DataFrame
            ladder_data = pd.concat([ladder_data,cur_ladder], ignore_index=True)
    return ladder_data


# Step 2: Read all CSV PF files and sample based on the rate
for file in sorted_filenames:
    ladder_data = pd.DataFrame()
    # print(file)
    cur_PF_res = pd.DataFrame(pd.read_csv(file_path + file))
    
    ladder_data = ladder_construction(file,rung_rates, delta_rate, cur_PF_res, ladder_data)
 
    all_ladders = pd.concat([all_ladders,ladder_data], ignore_index=True)

csv_filename = path + "All_TRQ_Ladder_075T025R.csv"
all_ladders.to_csv(csv_filename, index=False)

